# Chapter 0: Introduction to Engineering Software for Mechatronic Systems

---
### Python as a Language
- High-level, interpreted, dynamically typed
- Supports multiple paradigms: Procedural, Object-oriented, and Functional
- **Batteries included:** rich standard library
- Industry standard for robotics (ROS, simulation, data science)
- Readable syntax that encourages clean code

## 1. Encapsulation

Encapsulation bundles the data (reading) and methods (calibrate) together, while Abstraction hides the complex math inside a simple method call.

In [27]:
import random
import time

class Sensor:
    def __init__(self, name: str, offset: float = 0.5):
        self.name = name
        self.__offset = offset     # private calibration parameter
        self.__enabled = True      # private internal state

    def enable(self):
        self.__enabled = True

    def disable(self):
        self.__enabled = False

    def calibrate(self, new_offset: float):
        if not isinstance(new_offset, (int, float)):
            raise TypeError("Offset must be numeric")

        if abs(new_offset) > 10.0:
            raise ValueError("Calibration offset out of range")

        self.__offset = new_offset

    def get_calibrated_value(self, raw_value: float) -> float:
        if not self.__enabled:
            raise RuntimeError(f"Sensor {self.name} is disabled")

        return raw_value - self.__offset

In [28]:
#Test Cases 1
temp_sensor = Sensor("TMP36")
print(temp_sensor.get_calibrated_value(30.0))
print(temp_sensor.__dict__)
# Output: 29.5

#Test Cases 2: add offset parameter
temp_sensor = Sensor("TMP36", offset=1.0)
print(temp_sensor.get_calibrated_value(30.0))
print(temp_sensor.__dict__)
# Output: 29.0

#Test Cases 3: demonstrate encapsulation protection
temp_sensor = Sensor("TMP36", offset=1.0)
# Attempting to directly access private attribute (will raise AttributeError)
try:
    print(temp_sensor.__offset)
except AttributeError as e:
    print(f"Error: {e}")
# Attempting to modify private attribute directly (will not affect the real offset)
temp_sensor.__offset = 100 # ❌ creates a new attribute, does not change real offset
print(temp_sensor.get_calibrated_value(30.0))
# Output: 29.0
# Demonstrating Encapsulation Protection
print(temp_sensor.__dict__)


29.5
{'name': 'TMP36', '_Sensor__offset': 0.5, '_Sensor__enabled': True}
29.0
{'name': 'TMP36', '_Sensor__offset': 1.0, '_Sensor__enabled': True}
Error: 'Sensor' object has no attribute '__offset'
29.0
{'name': 'TMP36', '_Sensor__offset': 1.0, '_Sensor__enabled': True, '__offset': 100}


## 2. Inheritance

Inheritance allows a specific type of sensor to "inherit" the properties of a general sensor without rewriting code.

In [29]:
class UltrasonicSensor(Sensor):
    def __init__(self, name, pin):
        super().__init__(name) # Inherit from parent
        self.pin = pin

    def ping(self):
        return "Sending sonic pulse..."

dist_sensor = UltrasonicSensor("HC-SR04", 12)
print(dist_sensor.name) # Accessed from parent
print(dist_sensor.ping()) # Own unique method

HC-SR04
Sending sonic pulse...


## 2. Extend Inheritance Ultrasonics Example

In [ ]:

class UltrasonicSensor(Sensor):
    def __init__(self, name: str, pin: int):
        super().__init__(name)   # inherit name, offset, enabled
        self.pin = pin           # ultrasonic-specific hardware detail

    def ping(self) -> float:
        """
        Simulate sending an ultrasonic pulse and receiving echo time.
        """
        print(f"[{self.name}] Sending sonic pulse on pin {self.pin}...")
        time.sleep(0.05)

        # Simulated raw distance (cm)
        raw_distance = random.uniform(10.0, 200.0)
        print(f"[{self.name}] Echo received. Raw distance: {raw_distance:.2f} cm")
        return raw_distance

    def read(self) -> float:
        """
        High-level method combining:
        - sensor reading
        - calibration (inherited)
        """
        raw = self.ping()
        calibrated = self.get_calibrated_value(raw)
        return calibrated


In [ ]:
dist_sensor = UltrasonicSensor("HC-SR04", 12)

print(dist_sensor.name)            # inherited public attribute
print(dist_sensor.ping())          # ultrasonic-specific behavior

distance = dist_sensor.read()
print(f"Distance: {distance:.2f} cm")


HC-SR04
[HC-SR04] Sending sonic pulse on pin 12...
[HC-SR04] Echo received. Raw distance: 54.64 cm
54.64395980237526
[HC-SR04] Sending sonic pulse on pin 12...
[HC-SR04] Echo received. Raw distance: 50.17 cm
Distance: 49.67 cm


## 3. Polymorphism

Polymorphism allows different objects to be treated the same way. In mechatronics, you might want to "read" all sensors in a loop, regardless of whether they are Temperature, Pressure, or Light sensors.

In [38]:
class TemperatureSensor(Sensor):
    def __init__(self, name: str, adc_channel: int):
        super().__init__(name)
        self.adc_channel = adc_channel

    def read(self) -> float:
        raw_temp = random.uniform(20.0, 35.0)
        return self.get_calibrated_value(raw_temp)

In [41]:
def log_sensor(sensor: Sensor):
    value = sensor.read()
    print(f"[LOG] {sensor.name}: {value:.2f}")

sensors = [
    UltrasonicSensor("HC-SR04", pin=12),
    TemperatureSensor("TMP36", adc_channel=0),
]

for s in sensors:
    log_sensor(s)


AttributeError: 'UltrasonicSensor' object has no attribute 'read'

## Think of it like a CNC machine: 
You give it a high-level command like move_to(x, y), and you don't need to know the specific electrical pulses sent to the stepper motors, the current limits, or the internal PID loop logic. You only care about the result.
The Two Layers of Abstraction
- The Interface (Public): What the user sees and interacts with (the "What").
- The Implementation (Private): The complex logic hidden "under the hood" (the "How").

## Python Example: A Motor Controller

In this example, we use an Abstract Base Class (ABC). This acts as a blueprint or "contract" that ensures any type of motor we build later will have the same basic commands.

In [11]:
from abc import ABC, abstractmethod

# The Abstract Class (The Blueprint)
class Motor(ABC):
    @abstractmethod
    def set_speed(self, rpm):
        """Every motor MUST implement this method"""
        pass

    @abstractmethod
    def stop(self):
        """Every motor MUST implement this method"""
        pass

# The Concrete Implementation (The Hidden Complexity)
class StepperMotor(Motor):
    def set_speed(self, rpm):
        # Implementation details hidden from the user:
        # 1. Calculate pulse frequency
        # 2. Check current limits
        # 3. Write to GPIO pins
        print(f"Stepper Motor: Calculating steps for {rpm} RPM...")
        
    def stop(self):
        print("Stepper Motor: Cutting power to coils.")

# Using the Abstraction
# The user doesn't care HOW the motor spins, only that it HAS a set_speed method.
my_motor = StepperMotor()
my_motor.set_speed(1500)
my_motor.stop()

Stepper Motor: Calculating steps for 1500 RPM...
Stepper Motor: Cutting power to coils.


## Fully Abstraction Class

In [10]:
from abc import ABC, abstractmethod

# 1. THE ABSTRACTION (The Blueprint)
class Motor(ABC):
    @abstractmethod
    def set_speed(self, rpm):
        """Every motor MUST implement this method"""
        pass

    @abstractmethod
    def stop(self):
        """Every motor MUST implement this method"""
        pass

# 2. CONCRETE IMPLEMENTATION A: Stepper Motor
class StepperMotor(Motor):
    def set_speed(self, rpm):
        # Hidden complexity: Step logic
        print(f">>> [Hardware Call] Pulsing GPIO pins for {rpm} RPM.")
        print(f"Stepper Motor: Moving at {rpm} RPM.")
        
    def stop(self):
        print("Stepper Motor: Engaging holding torque and stopping.")

# 3. CONCRETE IMPLEMENTATION B: DC Motor
class DCMotor(Motor):
    def set_speed(self, rpm):
        # Hidden complexity: PWM (Pulse Width Modulation) logic
        voltage = rpm * 0.01 
        print(f">>> [Hardware Call] Setting PWM Duty Cycle to {voltage}V.")
        print(f"DC Motor: Spinning at {rpm} RPM.")
        
    def stop(self):
        print("DC Motor: Shorting terminals for dynamic braking.")

# --- EXECUTION (The Input) ---

print("--- Testing Stepper ---")
my_stepper = StepperMotor()
my_stepper.set_speed(1500)
my_stepper.stop()

print("\n--- Testing DC Motor ---")
my_dc = DCMotor()
my_dc.set_speed(3000)
my_dc.stop()

--- Testing Stepper ---
>>> [Hardware Call] Pulsing GPIO pins for 1500 RPM.
Stepper Motor: Moving at 1500 RPM.
Stepper Motor: Engaging holding torque and stopping.

--- Testing DC Motor ---
>>> [Hardware Call] Setting PWM Duty Cycle to 30.0V.
DC Motor: Spinning at 3000 RPM.
DC Motor: Shorting terminals for dynamic braking.
